# References


We are using an example SQLite database from:

  + https://www.sqlitetutorial.net/sqlite-sample-database/

## Schema Diagram



Here is the Entity Relationship Diagram (ERD) for this database:

![](https://www.sqlitetutorial.net/wp-content/uploads/2015/11/sqlite-sample-database-color.jpg)

# Setup




## Downloading the Database



Downloading the database file (then refresh the filesystem to see the "chinook.db" file):

In [ ]:
import os

DB_FILEPATH = "chinook.db"

def download_db(db_filepath=DB_FILEPATH):
    """using some terminal commands to download the .db file"""

    if os.path.isfile(db_filepath):
        print("DATABASE FILE EXISTS!")
    else:
        print("---------------")
        print("DOWNLOADING...")
        !wget https://www.sqlitetutorial.net/wp-content/uploads/2018/03/chinook.zip

        print("---------------")
        print("UNZIPPING...")
        !unzip "/content/chinook.zip"

        print("---------------")
        print("CLEANING UP...")
        !rm "/content/chinook.zip"


download_db()
assert os.path.isfile(DB_FILEPATH)

---------------
DOWNLOADING...
--2023-11-05 18:20:47--  https://www.sqlitetutorial.net/wp-content/uploads/2018/03/chinook.zip
Resolving www.sqlitetutorial.net (www.sqlitetutorial.net)... 172.67.172.250, 104.21.30.141, 2606:4700:3037::ac43:acfa, ...
Connecting to www.sqlitetutorial.net (www.sqlitetutorial.net)|172.67.172.250|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 305596 (298K) [application/zip]
Saving to: ‘chinook.zip’

chinook.zip         100%[===================>] 298.43K  --.-KB/s    in 0.03s   

2023-11-05 18:20:47 (8.60 MB/s) - ‘chinook.zip’ saved [305596/305596]

---------------
UNZIPPING...
Archive:  /content/chinook.zip
  inflating: chinook.db              
---------------
CLEANING UP...


## Connecting to the Database


Depending on the type of database (e.g. SQLite, MySQL, PostgreSQL), we would use a different strategy and set of packages to connect with it.

In this case we have a SQLite database, so we can use the `sqlite` module to interface with the database in a lower-level way, or various functions from `pandas` pacakge to interface with the database in a higher-level way.

### SQLite Connection

In this section, we demonstrate our ability to connect to the database using the `sqlite` module.

In [ ]:
import sqlite3

connection = sqlite3.connect(DB_FILEPATH)
connection.row_factory = sqlite3.Row
print("CONNECTION:", connection)

CONNECTION: <sqlite3.Connection object at 0x7a617f440140>


In [ ]:
cursor = connection.cursor()
print("CURSOR", cursor)

CURSOR <sqlite3.Cursor object at 0x7a617f434ac0>


Example question: "Who are all our customers? What are their names?"

In [ ]:
sql = """
    SELECT *
    FROM customers;
"""
results = cursor.execute(sql).fetchall()
print("RESULTS:", len(results))
print(results)

RESULTS: 59
[<sqlite3.Row object at 0x7a618c3469b0>, <sqlite3.Row object at 0x7a617f42e350>, <sqlite3.Row object at 0x7a617f42ed10>, <sqlite3.Row object at 0x7a617f46ded0>, <sqlite3.Row object at 0x7a617f46c580>, <sqlite3.Row object at 0x7a617f46d6f0>, <sqlite3.Row object at 0x7a617f46e110>, <sqlite3.Row object at 0x7a617f46d7e0>, <sqlite3.Row object at 0x7a617f46d480>, <sqlite3.Row object at 0x7a617f46d5d0>, <sqlite3.Row object at 0x7a617f46dc00>, <sqlite3.Row object at 0x7a617f46d120>, <sqlite3.Row object at 0x7a617f46dcc0>, <sqlite3.Row object at 0x7a617f46d540>, <sqlite3.Row object at 0x7a617f46d570>, <sqlite3.Row object at 0x7a617f46cd60>, <sqlite3.Row object at 0x7a617f46ccd0>, <sqlite3.Row object at 0x7a617f46cd90>, <sqlite3.Row object at 0x7a617f46ce20>, <sqlite3.Row object at 0x7a617f46cdf0>, <sqlite3.Row object at 0x7a617f46d240>, <sqlite3.Row object at 0x7a617f46d660>, <sqlite3.Row object at 0x7a617f46d600>, <sqlite3.Row object at 0x7a617f46d690>, <sqlite3.Row object at 0x7a

In [ ]:
for row in results[0:3]:
    print("-----")
    print(type(row)) #> sqlite3.Row
    print(row)
    print(row["FirstName"],row["LastName"])

-----
<class 'sqlite3.Row'>
Luís Gonçalves
-----
<class 'sqlite3.Row'>
Leonie Köhler
-----
<class 'sqlite3.Row'>
François Tremblay


In [ ]:
# these are the columns in the customers table
print(results[0].keys())

['CustomerId', 'FirstName', 'LastName', 'Company', 'Address', 'City', 'State', 'Country', 'PostalCode', 'Phone', 'Fax', 'Email', 'SupportRepId']


In [ ]:
dict(results[0])

{'CustomerId': 1,
 'FirstName': 'Luís',
 'LastName': 'Gonçalves',
 'Company': 'Embraer - Empresa Brasileira de Aeronáutica S.A.',
 'Address': 'Av. Brigadeiro Faria Lima, 2170',
 'City': 'São José dos Campos',
 'State': 'SP',
 'Country': 'Brazil',
 'PostalCode': '12227-000',
 'Phone': '+55 (12) 3923-5555',
 'Fax': '+55 (12) 3923-5566',
 'Email': 'luisg@embraer.com.br',
 'SupportRepId': 3}

In [ ]:
results[0]["FirstName"]

'Luís'

Roll your own dataframe:

In [ ]:
from pandas import DataFrame

records = [dict(row) for row in results]
results_df = DataFrame(records)
results_df.head()

,CustomerId,FirstName,LastName,Company,Address,City,State,Country,PostalCode,Phone,Fax,Email,SupportRepId
0,1,Luís,Gonçalves,Embraer - Empresa Brasileira de Aeronáutica S.A.,"Av. Brigadeiro Faria Lima, 2170",São José dos Campos,SP,Brazil,12227-000,+55 (12) 3923-5555,+55 (12) 3923-5566,luisg@embraer.com.br,3
1,2,Leonie,Köhler,None,Theodor-Heuss-Straße 34,Stuttgart,None,Germany,70174,+49 0711 2842222,None,leonekohler@surfeu.de,5
2,3,François,Tremblay,None,1498 rue Bélanger,Montréal,QC,Canada,H2G 1A7,+1 (514) 721-4711,None,ftremblay@gmail.com,3
3,4,Bjørn,Hansen,None,Ullevålsveien 14,Oslo,None,Norway,0171,+47 22 44 22 22,None,bjorn.hansen@yahoo.no,4
4,5,František,Wichterlová,JetBrains s.r.o.,Klanova 9/506,Prague,None,Czech Republic,14700,+420 2 4172 5555,+420 2 4172 5555,frantisekw@jetbrains.com,4


In [ ]:
results_df[["CustomerId", "FirstName", "LastName"]]

,CustomerId,FirstName,LastName
0,1,Luís,Gonçalves
1,2,Leonie,Köhler
2,3,François,Tremblay
3,4,Bjørn,Hansen
4,5,František,Wichterlová
5,6,Helena,Holý
6,7,Astrid,Gruber
7,8,Daan,Peeters
8,9,Kara,Nielsen
9,10,Eduardo,Martins


For this exercise, we'll prefer to use more specific SQL query to arrive at the answer more directly:



In [ ]:
sql = """
    SELECT DISTINCT CustomerId, firstName, lastName
    FROM customers;
"""
results = cursor.execute(sql).fetchall()
print("RESULTS:", len(results))
for row in results:
    print(row["CustomerId"], row["firstName"], row["lastName"])

RESULTS: 59
1 Luís Gonçalves
2 Leonie Köhler
3 François Tremblay
4 Bjørn Hansen
5 František Wichterlová
6 Helena Holý
7 Astrid Gruber
8 Daan Peeters
9 Kara Nielsen
10 Eduardo Martins
11 Alexandre Rocha
12 Roberto Almeida
13 Fernanda Ramos
14 Mark Philips
15 Jennifer Peterson
16 Frank Harris
17 Jack Smith
18 Michelle Brooks
19 Tim Goyer
20 Dan Miller
21 Kathy Chase
22 Heather Leacock
23 John Gordon
24 Frank Ralston
25 Victor Stevens
26 Richard Cunningham
27 Patrick Gray
28 Julia Barnett
29 Robert Brown
30 Edward Francis
31 Martha Silk
32 Aaron Mitchell
33 Ellie Sullivan
34 João Fernandes
35 Madalena Sampaio
36 Hannah Schneider
37 Fynn Zimmermann
38 Niklas Schröder
39 Camille Bernard
40 Dominique Lefebvre
41 Marc Dubois
42 Wyatt Girard
43 Isabelle Mercier
44 Terhi Hämäläinen
45 Ladislav Kovács
46 Hugh O'Reilly
47 Lucas Mancini
48 Johannes Van der Berg
49 Stanisław Wójcik
50 Enrique Muñoz
51 Joakim Johansson
52 Emma Jones
53 Phil Hughes
54 Steve Murray
55 Mark Taylor
56 Diego Gutiérrez
57

### Pandas Helper Functions

Or use the `pandas.read_sql_query()` function to get a dataframe:

In [ ]:
from pandas import read_sql_query

results_df = read_sql_query(sql, connection)
results_df.head()

,CustomerId,FirstName,LastName
0,1,Luís,Gonçalves
1,2,Leonie,Köhler
2,3,François,Tremblay
3,4,Bjørn,Hansen
4,5,František,Wichterlová


In [ ]:
read_sql_query("SELECT DISTINCT CustomerId, FirstName, LastName FROM customers;", connection)

,CustomerId,FirstName,LastName
0,1,Luís,Gonçalves
1,2,Leonie,Köhler
2,3,François,Tremblay
3,4,Bjørn,Hansen
4,5,František,Wichterlová
5,6,Helena,Holý
6,7,Astrid,Gruber
7,8,Daan,Peeters
8,9,Kara,Nielsen
9,10,Eduardo,Martins


# Challenges

Write SQL code to answer each of the questions in Parts 1-3. Execute the queries using Python.

> NOTE: in many cases it is possible to execute a simple SQL query to grab all the data, and then use pandas to further manipulate / process the data. HOWEVER: for this exercise, we are looking for you to **arrive at the answers using SQL only**! NO pandas, except for executing the queries.

Then also tackle Part 4, which asks you to setup a new database and practice storing some example data.


## Part 1 (Basic Queries)





A) Who are our customers (include their first and last names?)

B) Which customers are from the US (include their first and last names and email addresses)?

C) Which customers are from either the US or the UK (include their first and last names and email addresses)?

D) For all customers in the US or UK, sort by country, then state (HINT: use `ORDER BY` clause).


In [ ]:
sql = """

    SELECT CustomerId, FirstName, LastName
    FROM customers;

"""
read_sql_query(sql, connection)

,CustomerId,FirstName,LastName
0,1,Luís,Gonçalves
1,2,Leonie,Köhler
2,3,François,Tremblay
3,4,Bjørn,Hansen
4,5,František,Wichterlová
5,6,Helena,Holý
6,7,Astrid,Gruber
7,8,Daan,Peeters
8,9,Kara,Nielsen
9,10,Eduardo,Martins


In [ ]:
sql = """

    SELECT CustomerId, FirstName, LastName,  Email, Country
    FROM customers
    WHERE Country = 'USA';

"""
read_sql_query(sql, connection)


,CustomerId,FirstName,LastName,Email,Country
0,16,Frank,Harris,fharris@google.com,USA
1,17,Jack,Smith,jacksmith@microsoft.com,USA
2,18,Michelle,Brooks,michelleb@aol.com,USA
3,19,Tim,Goyer,tgoyer@apple.com,USA
4,20,Dan,Miller,dmiller@comcast.com,USA
5,21,Kathy,Chase,kachase@hotmail.com,USA
6,22,Heather,Leacock,hleacock@gmail.com,USA
7,23,John,Gordon,johngordon22@yahoo.com,USA
8,24,Frank,Ralston,fralston@gmail.com,USA
9,25,Victor,Stevens,vstevens@yahoo.com,USA


In [ ]:
sql = """

    SELECT CustomerId, FirstName, LastName,  Email, Country
    FROM customers
    WHERE Country = 'USA' OR Country = 'United Kingdom';

"""
read_sql_query(sql, connection)


,CustomerId,FirstName,LastName,Email,Country
0,16,Frank,Harris,fharris@google.com,USA
1,17,Jack,Smith,jacksmith@microsoft.com,USA
2,18,Michelle,Brooks,michelleb@aol.com,USA
3,19,Tim,Goyer,tgoyer@apple.com,USA
4,20,Dan,Miller,dmiller@comcast.com,USA
5,21,Kathy,Chase,kachase@hotmail.com,USA
6,22,Heather,Leacock,hleacock@gmail.com,USA
7,23,John,Gordon,johngordon22@yahoo.com,USA
8,24,Frank,Ralston,fralston@gmail.com,USA
9,25,Victor,Stevens,vstevens@yahoo.com,USA


In [ ]:
sql = """

    SELECT CustomerId, FirstName, LastName,  Email, Country, State
    FROM customers
    WHERE Country IN ('USA', 'United Kingdom')
    ORDER BY Country and State;

"""
read_sql_query(sql, connection)


,CustomerId,FirstName,LastName,Email,Country,State
0,16,Frank,Harris,fharris@google.com,USA,CA
1,17,Jack,Smith,jacksmith@microsoft.com,USA,WA
2,18,Michelle,Brooks,michelleb@aol.com,USA,NY
3,19,Tim,Goyer,tgoyer@apple.com,USA,CA
4,20,Dan,Miller,dmiller@comcast.com,USA,CA
5,21,Kathy,Chase,kachase@hotmail.com,USA,NV
6,22,Heather,Leacock,hleacock@gmail.com,USA,FL
7,23,John,Gordon,johngordon22@yahoo.com,USA,MA
8,24,Frank,Ralston,fralston@gmail.com,USA,IL
9,25,Victor,Stevens,vstevens@yahoo.com,USA,WI



## Part 2 (Aggregations)







A) How many customers do we have total?

B) How many customers are from the US?

C) For all US customers, how many are in each state?

D) How many customers in each country?

E) Which 5 countries have the most customers? Sort them in descending order of their customer count.


In [ ]:
sql = """

    SELECT COUNT(DISTINCT CustomerId) AS customer_count
    FROM customers;

"""
read_sql_query(sql, connection)


,customer_count
0,59


In [ ]:
sql = """

    SELECT COUNT(DISTINCT CustomerId) AS customer_count
    FROM customers
    WHERE country = 'USA';

"""
read_sql_query(sql, connection)


,customer_count
0,13


In [ ]:
sql = """

    SELECT State, COUNT(DISTINCT CustomerId) AS customer_count
    FROM customers
    GROUP BY State
    Having Country = 'USA'
    ORDER BY customer_count DESC;

"""
read_sql_query(sql, connection)


,State,customer_count
0,CA,3
1,WI,1
2,WA,1
3,UT,1
4,TX,1
5,NY,1
6,NV,1
7,MA,1
8,IL,1
9,FL,1


In [ ]:
sql = """

    SELECT Country, COUNT(DISTINCT CustomerId) AS customer_count
    FROM customers
    GROUP BY Country
    ORDER BY customer_count DESC;

"""
read_sql_query(sql, connection)


,Country,customer_count
0,USA,13
1,Canada,8
2,France,5
3,Brazil,5
4,Germany,4
5,United Kingdom,3
6,Portugal,2
7,India,2
8,Czech Republic,2
9,Sweden,1


In [ ]:
sql = """

    SELECT Country, COUNT(DISTINCT CustomerId) AS customer_count
    FROM customers
    GROUP BY Country
    ORDER BY customer_count DESC
    LIMIT 5;

"""
read_sql_query(sql, connection)


,Country,customer_count
0,USA,13
1,Canada,8
2,France,5
3,Brazil,5
4,Germany,4



## Part 3 (Multi-Table / Joins)



> HINT: here is a join types reference:
>
> <img src="https://camo.githubusercontent.com/52e1d3a424ed13892b4105190cd486f320ccc98b00ba33e40e36f678f23f6c9e/68747470733a2f2f7777772e696f6e6f732e636f6d2f6469676974616c67756964652f66696c6561646d696e2f4469676974616c47756964652f53637265656e73686f74735f323031382f4f757465722d4a6f696e2e6a7067" height=300 width=550/>






A)  For each album, what is the name of the artist that produced it? Expect 347 rows (row per album).

B) For each of the 275 artists, how many albums does each have? Optionally sort the results so the artists with the most albums are first.

C) For each of the 275 artists, how many tracks does each have? Optionally sort the results so the artists with the most tracks are first.

D) Who are the ten customers who have paid the most money (include their first and last names, as well as how many invoices each has had, as well as the total invoice amount for each)?



In [ ]:
sql = """

    SELECT ar.ArtistId, al.Title as album_title, ar.Name as artist_name
    FROM albums al
    JOIN artists ar
    ON al.ArtistId = ar.ArtistId

"""
read_sql_query(sql, connection)

,ArtistId,album_title,artist_name
0,1,For Those About To Rock We Salute You,AC/DC
1,2,Balls to the Wall,Accept
2,2,Restless and Wild,Accept
3,1,Let There Be Rock,AC/DC
4,3,Big Ones,Aerosmith
...,...,...,...
342,226,Respighi:Pines of Rome,Eugene Ormandy
343,272,Schubert: The Late String Quartets & String Qu...,Emerson String Quartet
344,273,Monteverdi: L'Orfeo,"C. Monteverdi, Nigel Rogers - Chiaroscuro; Lon..."
345,274,Mozart: Chamber Music,Nash Ensemble


In [ ]:
sql = """

    SELECT ar.ArtistId, ar.Name AS artist_name, COUNT(DISTINCT al.Title) as album_count
    FROM artists ar
    LEFT JOIN albums al
    ON ar.ArtistId = al.ArtistId
    GROUP BY ar.Name
    ORDER BY album_count DESC;

"""
read_sql_query(sql, connection)

,ArtistId,artist_name,album_count
0,90,Iron Maiden,21
1,22,Led Zeppelin,14
2,58,Deep Purple,11
3,150,U2,10
4,50,Metallica,10
...,...,...,...
270,26,Azymuth,0
271,166,Avril Lavigne,0
272,161,Aerosmith & Sierra Leone's Refugee Allstars,0
273,239,"Academy of St. Martin in the Fields, Sir Nevil...",0


In [ ]:
sql = """

    SELECT ar.ArtistId, ar.Name AS artist_name, COUNT(DISTINCT t.Name) as track_count
    FROM artists ar
    LEFT JOIN albums al
    ON ar.ArtistId = al.ArtistId
    LEFT JOIN tracks t
    ON al.AlbumId = t.AlbumId
    GROUP BY ar.Name
    ORDER BY track_count DESC;

"""
read_sql_query(sql, connection)

,ArtistId,artist_name,track_count
0,90,Iron Maiden,150
1,150,U2,124
2,50,Metallica,112
3,149,Lost,91
4,22,Led Zeppelin,91
...,...,...,...
270,26,Azymuth,0
271,166,Avril Lavigne,0
272,161,Aerosmith & Sierra Leone's Refugee Allstars,0
273,239,"Academy of St. Martin in the Fields, Sir Nevil...",0


In [ ]:
sql = """

    SELECT i.CustomerId, c.FirstName, c.LastName, c.Email, count(distinct i.InvoiceId) as Invoice_Count, sum(i.Total) as Total_Invoice
    FROM invoices i
    LEFT JOIN customers c
    ON i.CustomerId = c.CustomerId
    GROUP BY i.CustomerId
    ORDER BY Total_Invoice DESC
    LIMIT 10


"""
read_sql_query(sql, connection)

,CustomerId,FirstName,LastName,Email,Invoice_Count,Total_Invoice
0,6,Helena,Holý,hholy@gmail.com,7,49.62
1,26,Richard,Cunningham,ricunningham@hotmail.com,7,47.62
2,57,Luis,Rojas,luisrojas@yahoo.cl,7,46.62
3,45,Ladislav,Kovács,ladislav_kovacs@apple.hu,7,45.62
4,46,Hugh,O'Reilly,hughoreilly@apple.ie,7,45.62
5,28,Julia,Barnett,jubarnett@gmail.com,7,43.62
6,24,Frank,Ralston,fralston@gmail.com,7,43.62
7,37,Fynn,Zimmermann,fzimmermann@yahoo.de,7,43.62
8,7,Astrid,Gruber,astrid.gruber@apple.at,7,42.62
9,25,Victor,Stevens,vstevens@yahoo.com,7,42.62


## Part 4 (Database Management)




A) **Create your own example dataset**, formatted as a list of dictionaries, and store it in a variable called `records`. It should contain around ten records or so. Choose your own topic / domain. Can use ChatGPT to help generate the examples. Create a pandas dataframe of this data, and store in a variable called `records_df`.


B) **Create / connect to a new SQLite database** to store the data. Choose your own database file name, perhaps called "exercise.db".

C) **Create a new table** in the database, using a specified schema that matches the columns / fields in your example dataset. Choose your own table name that describes the kind of records we will be storing in it. The table's schema should ideally include an auto-incrementing integer primary key called `id`, as well as an auto-generated timestamp called `created_at`.

D) **Store the records** in the designated table.

E) Execute a query to **retrieve all records from the table**, and display them, to verify the data got stored properly.

Nice!



In [ ]:
# Importing libraries
import os
import sqlite3
from pandas import DataFrame, read_sql_query

In [ ]:
# Generating data
records = [
    {
        'company_name': 'Company A',
        'ticker': 'COMP_A',
        'monthly_avg_price': 100.45,
        'monthly_high': 105.20,
        'monthly_low': 95.30,
        'monthly_closing_price': 101.75,
    },
    {
        'company_name': 'Company B',
        'ticker': 'COMP_B',
        'monthly_avg_price': 45.60,
        'monthly_high': 50.10,
        'monthly_low': 41.80,
        'monthly_closing_price': 46.25,
    },
    {
        'company_name': 'Company C',
        'ticker': 'COMP_C',
        'monthly_avg_price': 75.90,
        'monthly_high': 80.60,
        'monthly_low': 70.25,
        'monthly_closing_price': 76.40,
    },
    {
        'company_name': 'Company D',
        'ticker': 'COMP_D',
        'monthly_avg_price': 120.75,
        'monthly_high': 125.50,
        'monthly_low': 115.20,
        'monthly_closing_price': 121.85,
    },
    {
        'company_name': 'Company E',
        'ticker': 'COMP_E',
        'monthly_avg_price': 55.30,
        'monthly_high': 60.20,
        'monthly_low': 50.10,
        'monthly_closing_price': 56.75,
    },
    {
        'company_name': 'Company F',
        'ticker': 'COMP_F',
        'monthly_avg_price': 90.15,
        'monthly_high': 95.30,
        'monthly_low': 85.40,
        'monthly_closing_price': 91.70,
    },
    {
        'company_name': 'Company G',
        'ticker': 'COMP_G',
        'monthly_avg_price': 70.60,
        'monthly_high': 75.80,
        'monthly_low': 65.90,
        'monthly_closing_price': 71.45,
    },
    {
        'company_name': 'Company H',
        'ticker': 'COMP_H',
        'monthly_avg_price': 55.80,
        'monthly_high': 60.40,
        'monthly_low': 52.70,
        'monthly_closing_price': 57.20,
    },
    {
        'company_name': 'Company I',
        'ticker': 'COMP_I',
        'monthly_avg_price': 105.20,
        'monthly_high': 110.80,
        'monthly_low': 100.10,
        'monthly_closing_price': 106.75,
    },
    {
        'company_name': 'Company J',
        'ticker': 'COMP_J',
        'monthly_avg_price': 75.40,
        'monthly_high': 80.60,
        'monthly_low': 70.90,
        'monthly_closing_price': 76.80,
    },
]

records_df = DataFrame(records)
records_df.head(10)

,company_name,ticker,monthly_avg_price,monthly_high,monthly_low,monthly_closing_price
0,Company A,COMP_A,100.45,105.2,95.30,101.75
1,Company B,COMP_B,45.60,50.1,41.80,46.25
2,Company C,COMP_C,75.90,80.6,70.25,76.40
3,Company D,COMP_D,120.75,125.5,115.20,121.85
4,Company E,COMP_E,55.30,60.2,50.10,56.75
5,Company F,COMP_F,90.15,95.3,85.40,91.70
6,Company G,COMP_G,70.60,75.8,65.90,71.45
7,Company H,COMP_H,55.80,60.4,52.70,57.20
8,Company I,COMP_I,105.20,110.8,100.10,106.75
9,Company J,COMP_J,75.40,80.6,70.90,76.80


In [ ]:
# Create / connect to a new SQLite database
DB_FILEPATH = "share_price.db"

class BaseDatabase:
    """A base interface into SQLite database."""

    def __init__(self, filepath=DB_FILEPATH):
        """Params:
            filepath (str) : path to the database that will be created
        """
        self.filepath = filepath
        print("------------------")
        print("DB FILEPATH:", os.path.abspath(self.filepath))

        self.connection = sqlite3.connect(self.filepath)
        self.connection.row_factory = sqlite3.Row
        print("CONNECTION:", self.connection)

        self.cursor = self.connection.cursor()
        print("CURSOR:", self.cursor)

    def insert_df(self, df:DataFrame, table_name:str): # auto_increment=False
        """Inserts data into a given table.
            If table does not exist, will create the table.
            Normally with a SQL-only solution we would need to create the table first, with a given schema.
            But pandas.DataFrame.to_sql infers the schema from the dataframe schema.

            Params:

                table_name (str) : name of table to insert data into

                df (DataFrame) : the data to save or append in the given table
        """
        # https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.to_sql.html
        df.to_sql(table_name, con=self.connection,
            if_exists="append", # append to existing tables (don't throw error)
            index=False
        )

    def query_to_df(self, sql:str) -> DataFrame:
        """Executes a query and returns the results as a dataframe"""
        return read_sql_query(sql, con=self.connection)


db = BaseDatabase()

------------------
DB FILEPATH: /content/share_price.db
CONNECTION: <sqlite3.Connection object at 0x7a52ef46c140>
CURSOR: <sqlite3.Cursor object at 0x7a52c11ef740>


In [ ]:
# Creating a table
db.cursor.execute("DROP TABLE IF EXISTS records;")
db.cursor.execute("""
    CREATE TABLE IF NOT EXISTS books (

        id                        INTEGER PRIMARY KEY,

        company_name              VARCHAR(255),
        ticker                    VARCHAR(255),
        monthly_avg_price         INTEGER,
        monthly_high              INTEGER,
        monthly_low               INTEGER,
        monthly_closing_price     INTEGER,

        created_at                DATETIME NOT NULL DEFAULT (strftime('%Y-%m-%d %H:%M:%f', 'now'))
    );
""")
db.connection.commit()

In [ ]:
# Save data into the Database
db.insert_df(df=records_df, table_name="records")

In [ ]:
# See if dat is stored
db.query_to_df("SELECT * FROM records;")

,company_name,ticker,monthly_avg_price,monthly_high,monthly_low,monthly_closing_price
0,Company A,COMP_A,100.45,105.2,95.30,101.75
1,Company B,COMP_B,45.60,50.1,41.80,46.25
2,Company C,COMP_C,75.90,80.6,70.25,76.40
3,Company D,COMP_D,120.75,125.5,115.20,121.85
4,Company E,COMP_E,55.30,60.2,50.10,56.75
5,Company F,COMP_F,90.15,95.3,85.40,91.70
6,Company G,COMP_G,70.60,75.8,65.90,71.45
7,Company H,COMP_H,55.80,60.4,52.70,57.20
8,Company I,COMP_I,105.20,110.8,100.10,106.75
9,Company J,COMP_J,75.40,80.6,70.90,76.80
